In [ ]:
case_ID = 0

from pathlib import Path
import gmsh
from mpi4py import MPI
from dolfinx.io import gmshio
import numpy as np
import matplotlib.pyplot as plt

gmsh.initialize()
gmsh.model.add("geometry")

L1, L2, L3, L4 = 2, 2., 2, 2.
h, w = 2., 2

p1 = gmsh.model.geo.addPoint(-L1/2, -L4/2, 0, meshSize=0.2)
p2 = gmsh.model.geo.addPoint(0, -h/2, 0, meshSize=0.2)
p3 = gmsh.model.geo.addPoint(L1/2, -L2/2, 0, meshSize=0.2)
p4 = gmsh.model.geo.addPoint(w/2, 0, 0, meshSize=0.2)
p5 = gmsh.model.geo.addPoint(L3/2, L2/2, 0, meshSize=0.2)
p6 = gmsh.model.geo.addPoint(0, h/2, 0, meshSize=0.2)
p7 = gmsh.model.geo.addPoint(-L3/2, L4/2, 0, meshSize=0.2)
p8 = gmsh.model.geo.addPoint(-w/2, 0, 0, meshSize=0.2)

l1 = gmsh.model.geo.add_bspline([p1, p2, p3])
l2 = gmsh.model.geo.add_bspline([p3, p4, p5])
l3 = gmsh.model.geo.add_bspline([p5, p6, p7])
l4 = gmsh.model.geo.add_bspline([p7, p8, p1])

curve_loop = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
surface = gmsh.model.geo.addPlaneSurface([curve_loop])

gmsh.model.geo.synchronize()

bottom_tag_num, left_tag_num, top_tag_num, right_tag_num, interior_tag_num = 1, 2, 3, 4, 5

gmsh.model.addPhysicalGroup(1, [l1], bottom_tag_num, name="bottom")
gmsh.model.addPhysicalGroup(1, [l2], left_tag_num, name="left")
gmsh.model.addPhysicalGroup(1, [l3], top_tag_num, name="top")
gmsh.model.addPhysicalGroup(1, [l4], right_tag_num, name="right")
gmsh.model.addPhysicalGroup(2, [surface], interior_tag_num, name="interior")

gmsh.model.mesh.generate(2)

mesh_folder = Path("./paraview") / f"case_ID_{case_ID}"
mesh_folder.mkdir(exist_ok=True, parents=True)
gmsh.write(str(mesh_folder) + "/mesh.vtk")

domain, cell_tags, facet_tags = gmshio.model_to_mesh(gmsh.model, MPI.COMM_WORLD, 0, gdim=2)

gmsh.finalize()

In [ ]:
from matplotlib.tri import Triangulation

points = domain.geometry.x
cells = domain.topology.connectivity(2, 0).array.reshape((-1, 3))

# Plot the mesh using Matplotlib
plt.figure(figsize=(8, 6))
triang = Triangulation(points[:, 0], points[:, 1], cells)
plt.triplot(triang, color="black")
plt.gca().set_aspect('equal')
plt.title("2D Geometry Mesh")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.show()

In [ ]:
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)

In [ ]:
from dolfinx import fem, default_scalar_type
import ufl
V = fem.functionspace(domain, ("Lagrange", 1))
x = ufl.SpatialCoordinate(domain)

u_ex = 1 + x[0]**2 * x[1] + 2 * x[0] * x[1]**2

uD = fem.Function(V)
uD.interpolate(fem.Expression(u_ex, V.element.interpolation_points()))
src_term = -(4* x[0] + 2 * x[1])


dofs_bottom_b = fem.locate_dofs_topological(V, fdim, facet_tags.find(bottom_tag_num))
dofs_left_b = fem.locate_dofs_topological(V, fdim, facet_tags.find(left_tag_num))
dofs_top_b = fem.locate_dofs_topological(V, fdim, facet_tags.find(top_tag_num))
dofs_right_b = fem.locate_dofs_topological(V, fdim, facet_tags.find(right_tag_num))

dofs_drchlt_BC = np.concatenate([dofs_left_b, dofs_right_b])
dofs_neumnn_BC = np.concatenate([dofs_bottom_b, dofs_top_b])

drchlt_bc = fem.dirichletbc(uD, dofs_drchlt_BC)


In [ ]:
n = ufl.FacetNormal(domain)
g_bottom = (x[0] ** 2 + 4 * x[0] * x[1])
g_top = -(x[0] ** 2 + 4 * x[0] * x[1])
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags)

In [ ]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [ ]:
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = src_term * v * ufl.dx - g_bottom * v * ds(bottom_tag_num) - g_top * v * ds(top_tag_num)

In [ ]:
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(
    a, L, bcs=[drchlt_bc], 
    petsc_options={"ksp_type": "preonly",
                   "pc_type": "lu",
                   "ksp_monitor": None})
uh = problem.solve()

In [ ]:
V2 = fem.functionspace(domain, ("Lagrange", 2))
uex = fem.Function(V2)
uex.interpolate(lambda x: 1 + x[0]**2 * x[1] + 2 * x[0] * x[1]**2)

L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = np.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

error_max = np.max(np.abs(uD.x.array-uh.x.array))
if domain.comm.rank == 0:
    print(f"Error_L2 : {error_L2:.2e}")
    print(f"Error_max : {error_max:.2e}")

In [ ]:
udiff = fem.Function(V)
udiff.vector.setArray(np.abs(uh.vector.getArray() - uD.vector.getArray()) / (uh.vector.getArray()).max())

In [ ]:
triang = Triangulation(points[:, 0], points[:, 1], cells)

plt.figure(figsize=(8, 6))
tripcolor_plot = plt.tripcolor(triang, uh.vector.getArray(), shading='gouraud')
plt.triplot(triang, color='black', linewidth=0.5)
plt.gca().set_aspect('equal')
plt.title("uh")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.colorbar(tripcolor_plot)
plt.show()

plt.figure(figsize=(8, 6))
tripcolor_plot = plt.tripcolor(triang, uD.vector.getArray(), shading='gouraud')
plt.triplot(triang, color='black', linewidth=0.5)
plt.gca().set_aspect('equal')
plt.title("uD")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.colorbar(tripcolor_plot)
plt.show()

plt.figure(figsize=(8, 6))
tripcolor_plot = plt.tripcolor(triang, udiff.vector.getArray(), shading='gouraud')
plt.triplot(triang, color='black', linewidth=0.5)
plt.gca().set_aspect('equal')
plt.title("udiff")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.colorbar(tripcolor_plot)
plt.show()

In [ ]:
drchlt_bool = np.zeros(points.shape[0])
neumnn_bool = np.zeros(points.shape[0])
drchlt_bool[dofs_drchlt_BC] = 1.
neumnn_bool[dofs_neumnn_BC] = 1.
data = np.concatenate(
    [points[:, 0].reshape((-1, 1)),
     points[:, 1].reshape((-1, 1)),
     drchlt_bool.reshape((-1, 1)),
     neumnn_bool.reshape((-1, 1)),
     uh.vector.getArray().reshape((-1, 1))], axis=1)

data_folder = Path("./data")
data_folder.mkdir(exist_ok=True, parents=True)
filename = data_folder / f"case_ID_{case_ID}"

np.save(filename, data)

In [ ]:
from dolfinx import io
results_folder = Path("./paraview") / f"case_ID_{case_ID}"
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / f"case_ID_{case_ID}"

with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(udiff)